# Time Series Analysis

A *time series* is a form of data consisting of the values of a function sampled across different points in time. Most commonly, this series is a sequence taken at equally spaced points in time, where each point is called an *observation*. In this notebook we look at working with a time series in the form of a Pandas DataFrame.

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline 

## Time Series in Pandas

The most basic kind of time series data in Pandas is a Series indexed by timestamps, which is often represented as Python strings or datetime values. 


To start off, we will create a simple time series based on a list of values and a corresponding list of dates. These will represent the *observations* in our series.

In [ ]:
# generate a set of dates and values
dates = []
values = []
for i in range(20):
    dates.append(datetime(2023, 10, i+1))
    # create a pattern in the values
    if i < 14:
        value = 5 + i
    else:
        value = 5 + (2*(20-i))
    values.append(value)

We will turn this into a Pandas Series, where the list of dates will become the index of the Series:

In [ ]:
ts = pd.Series(values, index=dates)
ts

Typically we will visualise this using a line plot:

In [ ]:
# generate a plot
p = ts.plot(figsize=(9, 4), fontsize=13, color="darkblue")
# customise the plot
p.set_ylim(0, 20)
p.set_xlabel("Date", fontsize=13)
p.set_ylabel("Value", fontsize=13);

A Pandas time series can be indexed and sliced in the same way as a normal Series. However, if we have a date type as an index for the series, we can do indexing and slicing based on specifying different dates. For exmaple:

In [ ]:
ts["2023-10-06"]

In [ ]:
ts["2023-10-04":"2023-10-06"]

In [ ]:
ts["2023-10-12":]

For longer time series, we can easily select slices of data for a specific month or year: 

In [ ]:
# create random series with 500 values using NumPy
lts = pd.Series(np.random.randn(500), index=pd.date_range('1/1/2021', periods=500))
print(lts.head())
print(lts.tail())

In [ ]:
lts["2022"].head()

In [ ]:
lts["2022-03"].head()

In [ ]:
# plot the series for a single month
p = lts["2022-03"].plot(figsize=(9, 4), fontsize=13, color="darkorange")
# customise the plot
p.set_xlabel("Date", fontsize=13)
p.set_ylabel("Value", fontsize=13);

Time series data is ordered chronologically, so we can slice with timestamps not contained in a time series:

In [ ]:
p = lts["2022-01":"2023-03"].plot(figsize=(9, 4),fontsize=13)

*Resampling* is the process of converting time series data from one frequency to another. This is done via the resample() function.

We can downsample - aggregate higher frequency data to a lower frequency:

In [ ]:
ts = pd.Series(np.random.randn(100), index=pd.date_range('1/1/2015', periods=100))
ts.head()

In [ ]:
# Convert from day frequency to month (M) frequency, by averaging values
ts_monthly = ts.resample("M").mean()

We can also upsample a time series, converting lower frequency to higher frequency data.

In [ ]:
ts = pd.Series(np.random.randn(5), index=pd.date_range('1/1/2015', periods=5))
ts

For example, we could upsample by hour (H). Note that the rows that are added in between have missing values (NaNs).

In [ ]:
ts_hourly = ts.resample('H').mean()
ts_hourly.head()

## Comparing Time Series

For some types of data we might have multiple time series that we want to compare over time (e.g. to see whether the series exhibit similar or different trends).

To demonstrate this, we will use an agricultural meat dataset (originally from the **ggplot** Python package). This dataset contains figures for livestock, dairy, and poultry production in the US over several decades.

We will load this data from a remote CSV file into a Pandas DataFrame:

In [ ]:
# note that we specify parse_dates to try to parse the index field (called "date") as a date.
df = pd.read_csv("http://mlg.ucd.ie/modules/COMP41680/agri-meat.csv", index_col="date", parse_dates=True)

We see that this dataset has one entry per month, from 1944 to 2012:

In [ ]:
df.head()

In [ ]:
df.tail()

We can produce a simple time series plot for the full dataset:

In [ ]:
p = df.plot(figsize=(11, 6), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);

We can also produce a plot for a specific time period:

In [ ]:
# Only plot data from 1980 to 2000
p = df["1980":"2000"].plot(figsize=(11, 6), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);

We can also look at a shorter time period, such as the months in a single year:

In [ ]:
p = df["1980-1":"1980-12"].plot(figsize=(11, 6), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);

Pandas has functionality for aggregating date and time based data. For example, we can group the data by year:

In [ ]:
# aggregate the sum of values for each year
df_year = df.groupby(df.index.year).sum()
df_year.head()

In [ ]:
p = df_year.plot(figsize=(12, 6), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13)
p.set_xlim(df_year.index.min(), df_year.index.max())
p.legend(fontsize=13)
p.xaxis.grid();

If we want to group the data by decade, we need to define a custom aggregation function which will take the year of a date and "floor" it - i.e. round it down to the nearest 10. So 1957 becomes 1950 etc.

In [ ]:
def to_decade(date_value):
    return (date_value.year // 10) * 10

In [ ]:
df_decade = df.groupby(to_decade).sum()

In [ ]:
df_decade

Let's plot a comparison of production across the decades:

In [ ]:
p = df_decade.plot(kind='bar',figsize=(12, 6), fontsize=13, zorder=3)
p.set_xlabel("Decade",fontsize=13)
p.set_ylabel("Production",fontsize=13)
p.legend(fontsize=13)
p.yaxis.grid();

We could also use stacked bars to look at total production volume per decade:

In [ ]:
p = df_decade.plot(kind='bar',figsize=(12, 6), fontsize=13, stacked=True, zorder=3)
p.set_xlabel("Decade",fontsize=13)
p.set_ylabel("Production",fontsize=13)
p.legend(fontsize=13)
p.yaxis.grid();

## Moving Averages

One way to extract a trend from a time series is to use a moving average. This divides the series into overlapping regions, called windows, and computes the average of the values in each window.

A *rolling mean* is a simple approach which computes the mean of the values in each window. The size of the window is the number of values it will include. Pandas provides a rolling_mean() function, which takes a Series and a window size and returns a new Series.

In [ ]:
# calculate and plot 3 year rolling mean beef production
rm = df["beef"].rolling(3).mean()
p = rm.plot(figsize=(12, 4.5), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);

In [ ]:
# calculate and plot 10 year rolling mean beef production
rm = df["beef"].rolling(10).mean()
p = rm.plot(figsize=(12, 4.5), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);

Increasing the window size produces a smoother plot, with less noisy. But be careful not to "over-smooth" the data:

In [ ]:
# calculate and plot 30 year rolling mean beef production
rm = df["beef"].rolling(30).mean()
p = rm.plot(figsize=(12, 4.5), fontsize=13)
p.set_xlabel("Date",fontsize=13)
p.set_ylabel("Production",fontsize=13);